# Ejecutabilidad de Intervenciones de Reconstrucción mediante Inversiones (IRI)

In [1]:
#Importar paquetes
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import sys
sys.path
sys.path.append('C:/Users/jchamba/Documents/Jannely/Proyecto12/unicode-2.9')
from unidecode import unidecode

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#Importamos variables temporales
now = datetime.now()
ayer = datetime.now() - timedelta(days=1)
anteayer = datetime.now() - timedelta(days=2)

# la fecha de corte es la más actual
fecha_corte = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y") # es la fecha actual en formato dd/mm/yyyy
print(now.strftime("%d") + '-' + now.strftime("%m") + '-' + now.strftime("%Y"))

03-05-2024


In [3]:
# Para limpiar variables duplicadas luego de un merge
def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

# Para crear el COD_ID de la ejecutora
def crear_id(df, NIVEL_GOB, SECTOR, PLIEGO, EJECUTORA):
    df = df.assign(COD_ID = np.where(df[NIVEL_GOB].str.startswith('2'), (df[PLIEGO].str.split('. ', n=1).str[0] +"-" + df[EJECUTORA].str.split('. ', n=1).str[0]),
                                     (df[SECTOR].str.split('. ', n=1).str[0] +"-" + df[PLIEGO].str.split('. ', n=1).str[0] +"-"+ df[EJECUTORA].str.split('. ', n=1).str[0])))
    return df

In [4]:
### Cargar bases
print(datetime.now())
bd_invierte = pd.read_csv(os.path.join(r'Y:\Data_CSV\BancoInv', 'InviertePe_03052024'+'.csv'),dtype={'CODIGO_UNICO': str, 'COD_ID': str, 'SEC_EJEC': str})
bd_seg1224 = pd.read_csv(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto14', 'SEGUIMIENTO_PANEL2_2012_2024_30042024'+'.csv'),dtype={'CODIGO_UNICO': str, 'COD_ID': str, 'SEC_EJEC': str})
bd_ejecutoras = pd.read_csv(os.path.join(r'Y:\Data_CSV\BancoInv\EjecutorasPptal', 'Ejecutoras_CUI_25042024'+'.csv'),dtype={'CODIGO_UNICO': str, 'COD_ID': str, 'SEC_EJEC': str})
bd_secejec = pd.read_excel(os.path.join(r'Y:\Data_CSV\Miscelanea', 'SEC_EJEC_UBIGEO' + '.xlsx'), dtype={'ID_PLIEGO': str, 'CODIGOSIAF': str, 'SEC_EJEC': str})
bd_iri = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto14', 'Data_iri_unificado_bd' + '.xlsx'), sheet_name='data', dtype={'ID_PLIEGO': str, 'CODIGO_UNICO': str, 'SEC_EJEC': str})
bd_ingresos = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Bases', 'CANON_190424_VF_sindecimal - GRGL' + '.xlsx'), sheet_name='Hoja3', dtype={'ID_PLIEGO': str, 'CUI': str, 'SEC_EJEC': str})
print(datetime.now())

2024-05-03 17:30:51.138995


C:\Users\jchamba\AppData\Local\Temp\ipykernel_11772\3236053565.py:3: DtypeWarning: Columns (2,3,17,25,33) have mixed types. Specify dtype option on import or set low_memory=False.
  bd_invierte = pd.read_csv(os.path.join(r'Y:\1) NUEVO compartido DAPT\Financiamiento_Data\Data_CSV\BancoInv', 'InviertePe_03052024'+'.csv'),dtype={'CODIGO_UNICO': str, 'COD_ID': str, 'SEC_EJEC': str})


2024-05-03 17:32:23.060941


In [94]:
bd_invierte.info()
bd_invierte.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518419 entries, 0 to 518418
Data columns (total 61 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CODIGO_UNICO             518419 non-null  object 
 1   COD_ID                   518399 non-null  object 
 2   COD_ID_INVIERTE          436007 non-null  object 
 3   CONSTRUIDO_COD_ID        436007 non-null  object 
 4   TIPO_PROYECTO            518419 non-null  object 
 5   SITUACION                518419 non-null  object 
 6   ESTADO                   518419 non-null  object 
 7   CERRADO                  508953 non-null  object 
 8   EXP_TCO                  439692 non-null  object 
 9   ET_VIGENTE               192863 non-null  object 
 10  COSTO_ACTUAL_BCO         518353 non-null  float64
 11  COSTO_ACTUALIZADO        518353 non-null  float64
 12  DEV_ACUM_AL2023          308140 non-null  float64
 13  FUNCION                  337059 non-null  object 
 14  DIVI

,CODIGO_UNICO,COD_ID,COD_ID_INVIERTE,CONSTRUIDO_COD_ID,TIPO_PROYECTO,SITUACION,ESTADO,CERRADO,EXP_TCO,ET_VIGENTE,...,FIN_EJECUCION_MES,MARCO,BENEFICIARIO,MONTO_VIABLE,PMI_0,PMI_1,PMI_2,PMI_3,ORIGEN,OBS_NRO
0,2280502,001-001,NaN,NaN,PI-SNIP,V,A,NO,SI,SI,...,NaN,S,24629.0,7396016.00,705868.0,104617.0,0.0,0.0,SPMIE2,2
1,2280502,447-001,NaN,NaN,PI-SNIP,V,A,NO,SI,SI,...,NaN,S,24629.0,7396016.00,705868.0,104617.0,0.0,0.0,SPMIE2,2
2,2318699,001-001,NaN,NaN,PI-SNIP,V,A,NO,SI,SI,...,NaN,S,19525.0,19865708.00,NaN,NaN,NaN,NaN,E2,1
3,2429179,001-001,NaN,NaN,PI-INVIERTE,V,A,NO,NO,NaN,...,NaN,I,2478.0,96866159.78,0.0,50000.0,50000.0,50000.0,SPMIE2,1
4,2430118,001-001,NaN,NaN,PI-INVIERTE,V,A,NO,SI,NO,...,NaN,I,92504.0,22166873.16,8409012.0,0.0,0.0,0.0,SPMIE2,2


In [95]:
secejec = bd_secejec.copy()
secejec.info()
secejec.head(2879)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NIVEL_GOB        2879 non-null   object 
 1   SEC_EJEC         2879 non-null   object 
 2   SECTOR           2879 non-null   object 
 3   PLIEGO           2879 non-null   object 
 4   EJECUTORA        2879 non-null   object 
 5   UBIGEO           2456 non-null   float64
 6   PLIEGO_SIAF      2879 non-null   object 
 7   DEPARTAMENTO     2456 non-null   object 
 8   PROVINCIA        1891 non-null   object 
 9   ID_INVIERTEPE    2879 non-null   object 
 10  PLIEGO_SIAF_RES  2879 non-null   object 
dtypes: float64(1), object(10)
memory usage: 247.5+ KB


,NIVEL_GOB,SEC_EJEC,SECTOR,PLIEGO,EJECUTORA,UBIGEO,PLIEGO_SIAF,DEPARTAMENTO,PROVINCIA,ID_INVIERTEPE,PLIEGO_SIAF_RES
0,1. GOBIERNO NACIONAL,3,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,003. SECRETARIA GENERAL - PCM,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
1,1. GOBIERNO NACIONAL,1677,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,017. AUTORIDAD PARA LA RECONSTRUCCIÓN CON CAMB...,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
2,1. GOBIERNO NACIONAL,1713,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,018. MEJORAMIENTO DE SERVICIOS A CIUDADANOS Y ...,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
3,1. GOBIERNO NACIONAL,1733,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,020. SECRETARIA TECNICA DE APOYO A LA COMISION...,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
4,1. GOBIERNO NACIONAL,1749,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,021. PROYECTO ESPECIAL LEGADO,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
...,...,...,...,...,...,...,...,...,...,...,...
2874,3. GOBIERNOS LOCALES,301864,25. UCAYALI,03. PADRE ABAD,04. MUNICIPALIDAD DISTRITAL DE NESHUYA,250304.0,250304. MUNICIPALIDAD DISTRITAL DE NESHUYA,25. UCAYALI,03. PADRE ABAD,SI,250304. MD DE NESHUYA
2875,3. GOBIERNOS LOCALES,301865,25. UCAYALI,03. PADRE ABAD,05. MUNICIPALIDAD DISTRITAL DE ALEXANDER VON H...,250305.0,250305. MUNICIPALIDAD DISTRITAL DE ALEXANDER V...,25. UCAYALI,03. PADRE ABAD,SI,250305. MD DE ALEXANDER VON HUMBOLDT
2876,3. GOBIERNOS LOCALES,301898,25. UCAYALI,03. PADRE ABAD,06. MUNICIPALIDAD DISTRITAL DE HUIPOCA,250306.0,250306. MUNICIPALIDAD DISTRITAL DE HUIPOCA,25. UCAYALI,03. PADRE ABAD,SI,250306. MD DE HUIPOCA
2877,3. GOBIERNOS LOCALES,301899,25. UCAYALI,03. PADRE ABAD,07. MUNICIPALIDAD DISTRITAL DE BOQUERON,250307.0,250307. MUNICIPALIDAD DISTRITAL DE BOQUERON,25. UCAYALI,03. PADRE ABAD,SI,250307. MD DE BOQUERON


In [96]:
invierte = bd_invierte.copy()
invierte.drop_duplicates(inplace=True)
invierte = invierte[['CODIGO_UNICO', 'ESTADO', 'SITUACION', 'TIPO_PROYECTO', 'EXP_TCO', 'FUNCION', 'COSTO_ACTUAL_BCO', 'DEV_ACUM_AL2023', 'POR_AVANCE_FISICO']]
invierte['ESTADO'] = invierte['ESTADO'].replace({'A': 'ACTIVO',
                                                 'C':'CERRADO',
                                                 'D': 'DESACTIVADO',
                                                 'Z':'FUERA DE INVIERTE'})
invierte.drop_duplicates(inplace=True)
invierte.info()
invierte.head(500585)

<class 'pandas.core.frame.DataFrame'>
Index: 501298 entries, 0 to 518418
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CODIGO_UNICO       501298 non-null  object 
 1   ESTADO             501298 non-null  object 
 2   SITUACION          501298 non-null  object 
 3   TIPO_PROYECTO      501298 non-null  object 
 4   EXP_TCO            428602 non-null  object 
 5   FUNCION            332422 non-null  object 
 6   COSTO_ACTUAL_BCO   501232 non-null  float64
 7   DEV_ACUM_AL2023    291022 non-null  float64
 8   POR_AVANCE_FISICO  60551 non-null   float64
dtypes: float64(3), object(6)
memory usage: 38.2+ MB


,CODIGO_UNICO,ESTADO,SITUACION,TIPO_PROYECTO,EXP_TCO,FUNCION,COSTO_ACTUAL_BCO,DEV_ACUM_AL2023,POR_AVANCE_FISICO
0,2280502,ACTIVO,V,PI-SNIP,SI,TRANSPORTE,15896362.25,15085876.82,94.58
2,2318699,ACTIVO,V,PI-SNIP,SI,AMBIENTE,26426259.05,7277438.20,28.58
3,2429179,ACTIVO,V,PI-INVIERTE,NO,TRANSPORTE,98946159.78,17500.00,NaN
4,2430118,ACTIVO,V,PI-INVIERTE,SI,AGROPECUARIA,33797778.23,17752913.09,43.45
6,2478903,CERRADO,V,PI-INVIERTE,SI,"PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA",1860869.70,1860869.70,53.18
...,...,...,...,...,...,...,...,...,...
517701,2646374,ACTIVO,V,PI-INVIERTE,NO,VIVIENDA Y DESARROLLO URBANO,678767.03,NaN,NaN
517702,2646375,ACTIVO,F,PI-INVIERTE,NaN,NaN,0.00,NaN,NaN
517703,2646376,ACTIVO,F,PI-INVIERTE,NaN,NaN,0.00,NaN,NaN
517704,2646377,ACTIVO,F,PI-INVIERTE,NaN,NaN,0.00,NaN,NaN


In [97]:
bd_seg1224.info()
bd_seg1224.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6751263 entries, 0 to 6751262
Data columns (total 73 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   COD_ID                 object 
 1   SEC_EJEC               object 
 2   COD_PRODUCTO_PROYECTO  int64  
 3   COD_ACT_OBRA           int64  
 4   COD_PROG_PPTAL         object 
 5   COD_FUENTE_RUBRO       object 
 6   COD_CADENA_FUNCIONAL   object 
 7   COD_CADENA_GASTO       object 
 8   PIA_2012               float64
 9   PIM_2012               float64
 10  CERTIFICADO_2012       float64
 11  COMPROMISO_ANUAL_2012  float64
 12  TOTAL_DEVENGADO_2012   float64
 13  PIA_2013               float64
 14  PIM_2013               float64
 15  CERTIFICADO_2013       float64
 16  COMPROMISO_ANUAL_2013  float64
 17  TOTAL_DEVENGADO_2013   float64
 18  PIA_2014               float64
 19  PIM_2014               float64
 20  CERTIFICADO_2014       float64
 21  COMPROMISO_ANUAL_2014  float64
 22  TOTAL_DEVENGADO_20

,COD_ID,SEC_EJEC,COD_PRODUCTO_PROYECTO,COD_ACT_OBRA,COD_PROG_PPTAL,COD_FUENTE_RUBRO,COD_CADENA_FUNCIONAL,COD_CADENA_GASTO,PIA_2012,PIM_2012,...,PIA_2023,PIM_2023,CERTIFICADO_2023,COMPROMISO_ANUAL_2023,TOTAL_DEVENGADO_2023,PIA_2024,PIM_2024,CERTIFICADO_2024,COMPROMISO_ANUAL_2024,TOTAL_DEVENGADO_2024
0,01-001-003,3,3000001,5000276,PP.0071,FR.1.00,03.006.0008,5.3.2.2.1.1,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-001-003,3,3000001,5000276,PP.0071,FR.1.00,03.006.0008,5.3.2.2.1.2,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-001-003,3,3000001,5000276,PP.0071,FR.1.00,03.006.0008,5.3.2.2.2.2,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-001-003,3,3000001,5000276,PP.0071,FR.1.00,03.006.0008,5.3.2.2.2.3,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-001-003,3,3000001,5000276,PP.0071,FR.1.00,03.006.0008,5.3.2.3.1.1,150000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
bd_seg1224.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6751263 entries, 0 to 6751262
Data columns (total 73 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   COD_ID                 object 
 1   SEC_EJEC               object 
 2   COD_PRODUCTO_PROYECTO  int64  
 3   COD_ACT_OBRA           int64  
 4   COD_PROG_PPTAL         object 
 5   COD_FUENTE_RUBRO       object 
 6   COD_CADENA_FUNCIONAL   object 
 7   COD_CADENA_GASTO       object 
 8   PIA_2012               float64
 9   PIM_2012               float64
 10  CERTIFICADO_2012       float64
 11  COMPROMISO_ANUAL_2012  float64
 12  TOTAL_DEVENGADO_2012   float64
 13  PIA_2013               float64
 14  PIM_2013               float64
 15  CERTIFICADO_2013       float64
 16  COMPROMISO_ANUAL_2013  float64
 17  TOTAL_DEVENGADO_2013   float64
 18  PIA_2014               float64
 19  PIM_2014               float64
 20  CERTIFICADO_2014       float64
 21  COMPROMISO_ANUAL_2014  float64
 22  TOTAL_DEVENGADO_20

In [99]:
seg1224 = bd_seg1224.copy()
seg1224.rename(columns={'COD_PRODUCTO_PROYECTO':'CODIGO_UNICO',
                       'TOTAL_DEVENGADO_2012': 'DEV_2012',
                       'TOTAL_DEVENGADO_2013': 'DEV_2013',
                       'TOTAL_DEVENGADO_2014': 'DEV_2014',
                       'TOTAL_DEVENGADO_2015': 'DEV_2015',
                       'TOTAL_DEVENGADO_2016': 'DEV_2016',
                       'TOTAL_DEVENGADO_2017': 'DEV_2017',
                       'TOTAL_DEVENGADO_2018': 'DEV_2018',
                       'TOTAL_DEVENGADO_2019': 'DEV_2019',
                       'TOTAL_DEVENGADO_2020': 'DEV_2020',
                       'TOTAL_DEVENGADO_2021': 'DEV_2021',
                       'TOTAL_DEVENGADO_2022': 'DEV_2022',
                       'TOTAL_DEVENGADO_2023': 'DEV_2023',
                       'TOTAL_DEVENGADO_2024': 'DEV_2024'}, inplace=True)
seg1224['CODIGO_UNICO'] = seg1224['CODIGO_UNICO'].astype(str)
seg1224.drop(columns=['COD_ID', 'SEC_EJEC', 'COD_ACT_OBRA', 'COD_PROG_PPTAL', 'COD_FUENTE_RUBRO', 'COD_CADENA_FUNCIONAL', 'COD_CADENA_GASTO'], inplace=True)
seg1224 = seg1224.groupby(['CODIGO_UNICO']).sum().reset_index()
seg1224.info()
seg1224.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264225 entries, 0 to 264224
Data columns (total 66 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CODIGO_UNICO           264225 non-null  object 
 1   PIA_2012               264225 non-null  float64
 2   PIM_2012               264225 non-null  float64
 3   CERTIFICADO_2012       264225 non-null  float64
 4   COMPROMISO_ANUAL_2012  264225 non-null  float64
 5   DEV_2012               264225 non-null  float64
 6   PIA_2013               264225 non-null  float64
 7   PIM_2013               264225 non-null  float64
 8   CERTIFICADO_2013       264225 non-null  float64
 9   COMPROMISO_ANUAL_2013  264225 non-null  float64
 10  DEV_2013               264225 non-null  float64
 11  PIA_2014               264225 non-null  float64
 12  PIM_2014               264225 non-null  float64
 13  CERTIFICADO_2014       264225 non-null  float64
 14  COMPROMISO_ANUAL_2014  264225 non-nu

,CODIGO_UNICO,PIA_2012,PIM_2012,CERTIFICADO_2012,COMPROMISO_ANUAL_2012,DEV_2012,PIA_2013,PIM_2013,CERTIFICADO_2013,COMPROMISO_ANUAL_2013,...,PIA_2023,PIM_2023,CERTIFICADO_2023,COMPROMISO_ANUAL_2023,DEV_2023,PIA_2024,PIM_2024,CERTIFICADO_2024,COMPROMISO_ANUAL_2024,DEV_2024
0,2000028,16882371.0,13086623.0,8788852.83,8726731.15,8683551.55,38786009.0,14770330.0,8.707656e+06,8.400107e+06,...,2347073.0,5822317.0,5396028.24,3754461.94,3754461.94,86000.0,2549531.0,1690642.40,1662974.44,678475.74
1,2000032,207283104.0,129541871.0,91504807.25,90133042.17,89965728.45,333347796.0,184485491.0,1.121454e+08,1.090006e+08,...,78541007.0,54373494.0,48473669.61,47089722.46,46944316.51,48584037.0,38501796.0,20196176.98,15327704.82,7489922.66
2,2000045,4940555.0,5759461.0,5412084.13,5412084.13,5412084.13,5161000.0,6206455.0,5.646969e+06,5.646969e+06,...,4820639.0,5854456.0,5648149.71,5099395.77,5099395.77,4611198.0,5780153.0,3387560.89,2085700.70,1350232.21
3,2000055,67165806.0,28990208.0,13202833.50,12821878.18,12816789.30,18990866.0,9948370.0,7.122897e+06,6.938819e+06,...,71794355.0,1123740.0,859623.95,693659.26,643659.26,73852193.0,29957186.0,546390.27,491787.52,400053.55
4,2000129,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000e+00,0.000000e+00,...,16836143.0,1456713.0,1218044.99,1214026.62,1214026.62,19165762.0,11065576.0,123155.57,96726.50,92626.16


In [100]:
segpim = bd_seg1224.copy()
segpim.rename(columns={'COD_PRODUCTO_PROYECTO':'CODIGO_UNICO'}, inplace=True)
segpim['CODIGO_UNICO'] = segpim['CODIGO_UNICO'].astype(str)
segpim = segpim[segpim['CODIGO_UNICO'].str[:1] == '2']
segpim = segpim[~(segpim["COD_FUENTE_RUBRO"].isin(["FR.1.00","FR.3.19"]))]
segpim = pd.merge(segpim, bd_ejecutoras[['CODIGO_UNICO','SEC_EJEC']], on=['CODIGO_UNICO'], how='left')
postmerge(segpim)
segpim = pd.merge(segpim, bd_secejec[['SEC_EJEC', 'PLIEGO_SIAF_RES']], on=['SEC_EJEC'], how='left')
postmerge(segpim)
segpim['COD_PLIEGO'] = segpim['PLIEGO_SIAF_RES'].str.split('.', n=1).str[0]
segpim['COD_PLIEGO']=segpim['COD_PLIEGO'].fillna(0)
segpim['PIM_2022']=segpim['PIM_2022'].fillna(0)
segpim['PIM_2023']=segpim['PIM_2023'].fillna(0)
segpim['PIM_2024']=segpim['PIM_2024'].fillna(0)
segpim = segpim.groupby(['COD_PLIEGO']).agg({'PIM_2022': 'sum', 'PIM_2023' : 'sum', 'PIM_2024' :'sum', 'TOTAL_DEVENGADO_2022':'sum', 
                                             'TOTAL_DEVENGADO_2023':'sum', 'TOTAL_DEVENGADO_2024':'sum'}).reset_index()
segpim.rename(columns={'PIM_2022': 'PIM_2022_Pliego',
                      'PIM_2023': 'PIM_2023_Pliego',
                      'PIM_2024': 'PIM_2024_Pliego',
                      'TOTAL_DEVENGADO_2022': 'EJEC_2022_Pliego',
                      'TOTAL_DEVENGADO_2023': 'EJEC_2023_Pliego', 
                      'TOTAL_DEVENGADO_2024': 'EJEC_2024_Pliego', }, inplace=True)
segpim.info()
segpim.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077 entries, 0 to 2076
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   COD_PLIEGO        2077 non-null   object 
 1   PIM_2022_Pliego   2077 non-null   float64
 2   PIM_2023_Pliego   2077 non-null   float64
 3   PIM_2024_Pliego   2077 non-null   float64
 4   EJEC_2022_Pliego  2077 non-null   float64
 5   EJEC_2023_Pliego  2077 non-null   float64
 6   EJEC_2024_Pliego  2077 non-null   float64
dtypes: float64(6), object(1)
memory usage: 113.7+ KB


,COD_PLIEGO,PIM_2022_Pliego,PIM_2023_Pliego,PIM_2024_Pliego,EJEC_2022_Pliego,EJEC_2023_Pliego,EJEC_2024_Pliego
0,0,0.0,0.0,0.0,0.00,0.00,0.00
1,01001,99140.0,0.0,0.0,98676.52,0.00,0.00
2,01002,0.0,0.0,0.0,0.00,0.00,0.00
3,010101,5120490.0,4283580.0,6579683.0,2768420.86,3184200.55,522607.69
4,010102,261340.0,159780.0,159780.0,251530.16,157997.50,0.00


In [101]:
ingresos = bd_ingresos.copy()
ingresos = crear_id(ingresos, 'NIVEL DE GOBIERNO', 'SECTOR', 'PLIEGO', 'UNIDAD EJECUTORA')

ingresos['COD_ID'] = np.where(ingresos['NIVEL DE GOBIERNO'].str.startswith('2'), (ingresos['SECTOR'].str.split('. ', n=1).str[0] + ingresos['PLIEGO'].str.split('. ', n=1).str[0]),
                                     (ingresos['SECTOR'].str.split('. ', n=1).str[0] + ingresos['PLIEGO'].str.split('. ', n=1).str[0] + ingresos['UNIDAD EJECUTORA'].str.split('. ', n=1).str[0]))

ingresos = ingresos.groupby(['COD_ID'])[['2025', '2026', '2027']].sum().reset_index()
ingresos.rename(columns={'2025':'PROY_2025',
                         '2026':'PROY_2026',
                         '2027':'PROY_2027'}, inplace=True)
ingresos.info()
ingresos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   COD_ID     1917 non-null   object 
 1   PROY_2025  1917 non-null   int64  
 2   PROY_2026  1917 non-null   float64
 3   PROY_2027  1917 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 60.0+ KB


,COD_ID,PROY_2025,PROY_2026,PROY_2027
0,010101,7395550,7457944.0,7627646.0
1,010102,538458,536930.0,532022.0
2,010103,543930,540882.0,534555.0
3,010104,538458,536930.0,532022.0
4,010105,538458,536930.0,532022.0


In [102]:
bd_secejec.info()
bd_secejec.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NIVEL_GOB        2879 non-null   object 
 1   SEC_EJEC         2879 non-null   object 
 2   SECTOR           2879 non-null   object 
 3   PLIEGO           2879 non-null   object 
 4   EJECUTORA        2879 non-null   object 
 5   UBIGEO           2456 non-null   float64
 6   PLIEGO_SIAF      2879 non-null   object 
 7   DEPARTAMENTO     2456 non-null   object 
 8   PROVINCIA        1891 non-null   object 
 9   ID_INVIERTEPE    2879 non-null   object 
 10  PLIEGO_SIAF_RES  2879 non-null   object 
dtypes: float64(1), object(10)
memory usage: 247.5+ KB


,NIVEL_GOB,SEC_EJEC,SECTOR,PLIEGO,EJECUTORA,UBIGEO,PLIEGO_SIAF,DEPARTAMENTO,PROVINCIA,ID_INVIERTEPE,PLIEGO_SIAF_RES
0,1. GOBIERNO NACIONAL,3,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,003. SECRETARIA GENERAL - PCM,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
1,1. GOBIERNO NACIONAL,1677,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,017. AUTORIDAD PARA LA RECONSTRUCCIÓN CON CAMB...,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
2,1. GOBIERNO NACIONAL,1713,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,018. MEJORAMIENTO DE SERVICIOS A CIUDADANOS Y ...,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
3,1. GOBIERNO NACIONAL,1733,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,020. SECRETARIA TECNICA DE APOYO A LA COMISION...,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS
4,1. GOBIERNO NACIONAL,1749,01. PRESIDENCIA CONSEJO MINISTROS,001. PRESIDENCIA DEL CONSEJO DE MINISTROS,021. PROYECTO ESPECIAL LEGADO,NaN,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS,NaN,NaN,SI,01001. PRESIDENCIA DEL CONSEJO DE MINISTROS


In [103]:
iri= bd_iri.copy()

iri.rename(columns={'ENTIDAD_PCM':'ENTIDAD_EJECUTORA'}, inplace=True)
iri = iri[['CODIGO_UNICO', 'ENTIDAD_EJECUTORA', 'NIVEL_GOB_PCM', 'ESTADO_PCM', 'PLIEGO_SIAF_RES', 'NIVEL_GOB_VAL']]
iri['ESTADO_PCM'] = iri['ESTADO_PCM'].replace({'1. CULMINADO // 7. PENDIENTE':'1. CULMINADO',
                                                 '3. EN PROCESO DE CONTRATACIÓN // 1. CULMINADO': '1. CULMINADO'})

iri = iri[iri['ESTADO_PCM'] != '1. CULMINADO'].reset_index(drop=True)

iri['NOMBRE_PLIEGO'] = np.where(iri['PLIEGO_SIAF_RES'].notna(), iri['PLIEGO_SIAF_RES'].str.split('.', n=1).str[1], 
                                iri['PLIEGO_SIAF_RES'])

caracteres_a_reemplazar = {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
for caracter, reemplazo in caracteres_a_reemplazar.items():
    iri['ENTIDAD_EJECUTORA'] = iri['ENTIDAD_EJECUTORA'].str.replace(caracter, reemplazo)
    
iri.NOMBRE_PLIEGO = iri.NOMBRE_PLIEGO.str.replace(' GORE',' GOBIERNO REGIONAL DE')
iri.NOMBRE_PLIEGO = iri.NOMBRE_PLIEGO.str.replace(' MD',' MUNICIPALIDAD DISTRITAL')
iri.NOMBRE_PLIEGO = iri.NOMBRE_PLIEGO.str.replace(' MP',' MUNICIPALIDAD PROVINCIAL')
iri.NOMBRE_PLIEGO = iri.NOMBRE_PLIEGO.str.replace(' MM',' MUNICIPALIDAD METROPOLITANA')
iri.NOMBRE_PLIEGO = iri.NOMBRE_PLIEGO.str.replace(' M.',' MINISTERIO')
iri.NOMBRE_PLIEGO = iri.NOMBRE_PLIEGO.str.replace(' -', ' - ')
iri.ENTIDAD_EJECUTORA = iri.ENTIDAD_EJECUTORA.str.replace('KAÑARIS', 'CAÑARIS')
iri['ENTIDAD_EJECUTORA'] = iri['ENTIDAD_EJECUTORA'].replace({'MINISTERIO DE AGRICULTURA Y RIEGO': 'MINISTERIO DE DESARROLLO AGRARIO Y RIEGO',
                                                               'MUNICIPALIDAD DISTRITAL SANTA CRUZ DE FLORES': 'MUNICIPALIDAD DISTRITAL DE SANTA CRUZ DE FLORES',
                                                               'MUNICIPALIDAD DISTRITAL LA UNION':'MUNICIPALIDAD DISTRITAL DE LA UNION',
                                                               'GOBIERNO REGIONAL DEL DEPARTAMENTO DE TUMBES':'GOBIERNO REGIONAL DE TUMBES',
                                                               'GOBIERNO REGIONAL DE PIURA-GERENCIA SUB REGIONAL MORROPON HUANCABAMBA': 'GOBIERNO REGIONAL DE PIURA',
                                                               'MUNICIPALIDAD PROVINCIAL GRAN CHIMU - CASCAS':'MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS',
                                                               'MUNICIPALIDAD DISTRITAL ASCENSION':'MUNICIPALIDAD DISTRITAL DE ASCENSION',
                                                              'MUNICIPALIDAD DISTRITAL DE LURIGANCHO (CHOSICA)': 'MUNICIPALIDAD DISTRITAL DE LURIGANCHO',
                                                              'MUNICIPALIDAD PROVINCIAL DEL SANTA - CHIMBOTE': 'MUNICIPALIDAD PROVINCIAL DE SANTA - CHIMBOTE',
                                                              'MUNICIPALIDAD DISTRITAL CANOAS DE PUNTA SAL': 'MUNICIPALIDAD DISTRITAL DE CANOAS DE PUNTA SAL',
                                                              'MUNICIPALIDAD PROVINCIAL DE MORROPON -CHULUCANAS': 'MUNICIPALIDAD PROVINCIAL DE MORROPON - CHULUCANAS',
                                                             'MUNICIPALIDAD PROVINCIAL DE MORROPON': 'MUNICIPALIDAD PROVINCIAL DE MORROPON - CHULUCANAS',
                                                              'MUNICIPALIDAD PROVINCIAL DEL ALTO AMAZONAS - YURIMAGUAS':'MUNICIPALIDAD PROVINCIAL DE ALTO AMAZONAS - YURIMAGUAS',
                                                              'MUNICIPALIDAD PROVINCIAL DE CAÑETE -SAN VICENTE DE CAÑETE': 'MUNICIPALIDAD PROVINCIAL DE CAÑETE - SAN VICENTE DE CAÑETE',
                                                             'MUNICIPALIDAD PROVINCIAL DE CAÑETE': 'MUNICIPALIDAD PROVINCIAL DE CAÑETE - SAN VICENTE DE CAÑETE',
                                                             'MUNICIPALIDAD PROVINCIAL DE CARLOS FERMIN FITZCARRALD':'MUNICIPALIDAD PROVINCIAL DE CARLOS F. FITZCARRALD - SAN LUIS',
                                                              'MUNICIPALIDAD PROVINCIAL DE CARLOS F. FITZCARRALD -SAN LUIS':'MUNICIPALIDAD PROVINCIAL DE CARLOS F. FITZCARRALD - SAN LUIS',
                                                             'MUNICIPALIDAD PROVINCIAL DE CONTRALMIRANTE VILLAR': 'MUNICIPALIDAD PROVINCIAL DE CONTRALMIRANTE VILLAR - ZORRITOS'
                                                              })
iri['NOMBRE_PLIEGO'] = iri['NOMBRE_PLIEGO'].str[1:]
iri['VALIDACION_PLIEGO']= np.where((iri['ENTIDAD_EJECUTORA']==iri['NOMBRE_PLIEGO']), 'SI', 'NO')
iri.drop(columns=['NOMBRE_PLIEGO'], inplace=True)
iri.info()
iri.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6475 entries, 0 to 6474
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CODIGO_UNICO       6475 non-null   object
 1   ENTIDAD_EJECUTORA  1235 non-null   object
 2   NIVEL_GOB_PCM      1235 non-null   object
 3   ESTADO_PCM         1235 non-null   object
 4   PLIEGO_SIAF_RES    6473 non-null   object
 5   NIVEL_GOB_VAL      6475 non-null   object
 6   VALIDACION_PLIEGO  6475 non-null   object
dtypes: object(7)
memory usage: 354.2+ KB


,CODIGO_UNICO,ENTIDAD_EJECUTORA,NIVEL_GOB_PCM,ESTADO_PCM,PLIEGO_SIAF_RES,NIVEL_GOB_VAL,VALIDACION_PLIEGO
0,2214514,MUNICIPALIDAD DISTRITAL DE HUARMACA,GOBIERNO LOCAL,2. EN EJECUCIÓN,200304. MD DE HUARMACA,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI
1,2276731,MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS,GOBIERNO LOCAL,2. EN EJECUCIÓN,131101. MP DE GRAN CHIMU -CASCAS,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI
2,2280490,MUNICIPALIDAD DISTRITAL DE HUANCHACO,GOBIERNO LOCAL,3. EN PROCESO DE CONTRATACIÓN,130104. MD DE HUANCHACO,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI
3,2286734,MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS,GOBIERNO LOCAL,2. EN EJECUCIÓN,131101. MP DE GRAN CHIMU -CASCAS,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI
4,2307424,MUNICIPALIDAD PROVINCIAL DE CUTERVO,GOBIERNO LOCAL,2. EN EJECUCIÓN,060601. MP DE CUTERVO,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI


In [104]:
iri1 = pd.merge(iri, bd_ejecutoras[['CODIGO_UNICO','SEC_EJEC']], on=['CODIGO_UNICO'], how='left')
postmerge(iri1)

iri2 = pd.merge(iri1, bd_secejec[['SEC_EJEC', 'NIVEL_GOB', 'SECTOR', 'PLIEGO', 'EJECUTORA', 'DEPARTAMENTO', 'PLIEGO_SIAF']], on=['SEC_EJEC'], how='left')
postmerge(iri2)

iri2['COD_ID'] = np.where(iri2['PLIEGO_SIAF_RES'].notna(), iri2['PLIEGO_SIAF_RES'].str.split('.', n=1).str[0], '0')

iri3 = pd.merge(iri2, invierte, on=['CODIGO_UNICO'], how='left')
postmerge(iri3)

iri4 = pd.merge(iri3, seg1224, on=['CODIGO_UNICO'], how='left')
postmerge(iri4)

iri4['Sum_PIM'] = iri4[['PIM_2021', 'PIM_2022', 'PIM_2023']].sum(axis=1)
iri4['%Avance_Ejec']= iri4['DEV_ACUM_AL2023']/ iri4['COSTO_ACTUAL_BCO']
iri4['Pendiente_Financiar'] = iri4['COSTO_ACTUAL_BCO'].fillna(0) - iri7['DEV_ACUM_AL2023'].fillna(0) - iri5['PIM_2024'].fillna(0)
iri4['Pendiente_Financiar'] = np.where(iri4['Pendiente_Financiar'].fillna(0)<=0, 0, iri7['Pendiente_Financiar'].fillna(0))

iri4['ESTADO_PCM']= np.where((iri4['ESTADO_PCM'].isna())&(iri4['COMPROMISO_ANUAL_2023'].fillna(0)>0), '2. EN EJECUCIÓN',
                             np.where((iri4['ESTADO_PCM'].isna())&(iri4['COMPROMISO_ANUAL_2023'].fillna(0)<=0)&(iri4['CERTIFICADO_2023'].fillna(0)>0), '3. EN PROCESO DE CONTRATACIÓN',
                                      np.where((iri4['ESTADO_PCM'].isna())&(iri4['DEV_2023'].fillna(0)==0)&(iri4['CERTIFICADO_2023'].fillna(0)==0), '5. CON ESTUDIOS',
                                               np.where((iri4['ESTADO_PCM'].isna())&((iri4['PIA_2024'].fillna(0)>0)|(iri4['PIM_2024'].fillna(0)>0)), '2. EN EJECUCIÓN',
                                                        np.where((iri4['ESTADO_PCM'].isna())&(iri4['CERTIFICADO_2023'].fillna(0)==0)&(iri4['EXP_TCO']=='SI'), '5. CON ESTUDIOS',
                                                                 np.where((iri4['ESTADO_PCM'].isna())&(iri4['CERTIFICADO_2023'].fillna(0)==0)&(iri4['EXP_TCO']=='NO'), '7. PENDIENTE',
                                                                          np.where((iri4['ESTADO_PCM'].isna())&(iri4['%Avance_Ejec'].fillna(0)>=1.0), '1. CULMINADO',
                                                                                   np.where((iri4['ESTADO_PCM'].isna())&(iri4['Pendiente_Financiar'].fillna(0)<=0), '1. CULMINADO',
                                                                                            np.where((iri4['ESTADO_PCM'].isna())&(iri4['%Avance_Ejec'].fillna(0)>=0.90)&(iri4['Sum_PIM'].fillna(0)== 0), 'CULMINADO(SIN CONTINUIDAD)',
                                                                                                     iri4['ESTADO_PCM'])))))))))

iri4 = iri4[iri4['ESTADO_PCM'] != '1. CULMINADO'].reset_index(drop=True)

iri4['PIM_2022'] = iri4['PIM_2022'].fillna(0)
iri4['PIM_2023'] = iri4['PIM_2023'].fillna(0)
iri4['PIM_2024'] = iri4['PIM_2024'].fillna(0)
iri4.rename(columns={'ESTADO_PCM':'ESTADO SITUACIONAL'}, inplace=True)

iri5 = pd.merge(iri4, ingresos, on=['COD_ID'], how='left')
postmerge(iri5)

iri5['COD_PLIEGO'] = iri5['PLIEGO_SIAF_RES'].str.split('.', n=1).str[0]

iri6 = pd.merge(iri5, segpim, on=['COD_PLIEGO'], how='left')
postmerge(iri6)
    
iri6.drop(columns=['NIVEL_GOB', 'SECTOR', 'PLIEGO', 'EJECUTORA', 'COD_ID', 'PIA_2012', 'PIM_2012', 'CERTIFICADO_2012', 'COMPROMISO_ANUAL_2012', 'DEV_2012', 'PIA_2013', 'PIM_2013', 'CERTIFICADO_2013', 
                  'COMPROMISO_ANUAL_2013', 'DEV_2013', 'PIA_2014', 'PIM_2014',	'CERTIFICADO_2014', 'COMPROMISO_ANUAL_2014', 'DEV_2014', 'PIA_2015', 'PIM_2015', 'CERTIFICADO_2015', 'COMPROMISO_ANUAL_2015', 'DEV_2015',
                  'PIA_2016', 'PIM_2016', 'CERTIFICADO_2016', 'COMPROMISO_ANUAL_2016', 'DEV_2016', 'PIA_2017', 'PIM_2017', 'CERTIFICADO_2017', 'COMPROMISO_ANUAL_2017',	'DEV_2017',	'PIA_2018',	'PIM_2018',	'CERTIFICADO_2018',
                  'COMPROMISO_ANUAL_2018',	'DEV_2018',	'PIA_2019',	'PIM_2019',	'CERTIFICADO_2019',	'COMPROMISO_ANUAL_2019', 'DEV_2019', 'PIA_2020', 'PIM_2020', 'CERTIFICADO_2020', 'COMPROMISO_ANUAL_2020', 'DEV_2020', 'Sum_PIM', 'COD_PLIEGO', '%Avance_Ejec'], inplace=True)
iri6.info()
iri6.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6476 entries, 0 to 6475
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CODIGO_UNICO           6476 non-null   object 
 1   ENTIDAD_EJECUTORA      1235 non-null   object 
 2   NIVEL_GOB_PCM          1235 non-null   object 
 3   ESTADO SITUACIONAL     6476 non-null   object 
 4   PLIEGO_SIAF_RES        6474 non-null   object 
 5   NIVEL_GOB_VAL          6476 non-null   object 
 6   VALIDACION_PLIEGO      6476 non-null   object 
 7   SEC_EJEC               6461 non-null   object 
 8   DEPARTAMENTO           4955 non-null   object 
 9   PLIEGO_SIAF            6461 non-null   object 
 10  ESTADO                 6476 non-null   object 
 11  SITUACION              6476 non-null   object 
 12  TIPO_PROYECTO          6476 non-null   object 
 13  EXP_TCO                4882 non-null   object 
 14  FUNCION                4905 non-null   object 
 15  COST

,CODIGO_UNICO,ENTIDAD_EJECUTORA,NIVEL_GOB_PCM,ESTADO SITUACIONAL,PLIEGO_SIAF_RES,NIVEL_GOB_VAL,VALIDACION_PLIEGO,SEC_EJEC,DEPARTAMENTO,PLIEGO_SIAF,...,Pendiente_Financiar,PROY_2025,PROY_2026,PROY_2027,PIM_2022_Pliego,PIM_2023_Pliego,PIM_2024_Pliego,EJEC_2022_Pliego,EJEC_2023_Pliego,EJEC_2024_Pliego
0,2214514,MUNICIPALIDAD DISTRITAL DE HUARMACA,GOBIERNO LOCAL,2. EN EJECUCIÓN,200304. MD DE HUARMACA,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301551,20. PIURA,200304. MUNICIPALIDAD DISTRITAL DE HUARMACA,...,67032.49,31203456.0,31069671.0,31322754.0,27772130.0,21128162.0,24606297.0,18801155.12,15156831.65,2866854.20
1,2276731,MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS,GOBIERNO LOCAL,2. EN EJECUCIÓN,131101. MP DE GRAN CHIMU -CASCAS,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301205,13. LA LIBERTAD,131101. MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU...,...,2514805.80,9873857.0,10212227.0,10638261.0,11869644.0,8417694.0,11053119.0,9353170.54,2375728.52,1021770.44
2,2280490,MUNICIPALIDAD DISTRITAL DE HUANCHACO,GOBIERNO LOCAL,3. EN PROCESO DE CONTRATACIÓN,130104. MD DE HUANCHACO,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301130,13. LA LIBERTAD,130104. MUNICIPALIDAD DISTRITAL DE HUANCHACO,...,0.80,12540314.0,12814030.0,13066998.0,10902775.0,14829780.0,16293690.0,6953196.79,3512579.09,1045555.06
3,2286734,MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS,GOBIERNO LOCAL,2. EN EJECUCIÓN,131101. MP DE GRAN CHIMU -CASCAS,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301205,13. LA LIBERTAD,131101. MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU...,...,246703.77,9873857.0,10212227.0,10638261.0,11869644.0,8417694.0,11053119.0,9353170.54,2375728.52,1021770.44
4,2307424,MUNICIPALIDAD PROVINCIAL DE CUTERVO,GOBIERNO LOCAL,2. EN EJECUCIÓN,060601. MP DE CUTERVO,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,300605,06. CAJAMARCA,060601. MUNICIPALIDAD PROVINCIAL DE CUTERVO,...,51568.34,34946256.0,35518065.0,36421860.0,37394289.0,33499686.0,21394906.0,22021816.14,28502524.73,4700287.59


In [105]:
#
iri7=iri6.copy()
iri7['EJEC_2022_Pliego']=iri7['EJEC_2022_Pliego'].fillna(0)
iri7['EJEC_2023_Pliego']=iri7['EJEC_2023_Pliego'].fillna(0)
iri7['EJEC_2024_Pliego']=iri7['EJEC_2024_Pliego'].fillna(0)
iri7['PIM_2022_Pliego']=iri7['PIM_2022_Pliego'].fillna(0)
iri7['PIM_2023_Pliego']=iri7['PIM_2023_Pliego'].fillna(0)
iri7['PIM_2024_Pliego']=iri7['PIM_2024_Pliego'].fillna(0)

iri7['Prom_EJEC_Pliego']= iri7[['EJEC_2022_Pliego', 'EJEC_2023_Pliego', 'EJEC_2024_Pliego']].mean(axis=1)
iri7['Prom_PIM_Pliego']= iri7[['PIM_2022_Pliego', 'PIM_2023_Pliego', 'PIM_2024_Pliego']].mean(axis=1)
iri7['Pendiente_Financiar_Pliego'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['Pendiente_Financiar'].transform('sum'))

#Número de Proyectos en Ejecución
iri7['N_Proyec_Pliego'] = np.where((iri7['Pendiente_Financiar'] >= 0), 1, 0)
iri7['N_Proyec_Ejecucion'] = np.where((iri7['Pendiente_Financiar'] >= 0)&(iri7['ESTADO SITUACIONAL']=='2. EN EJECUCIÓN'), 1, 0) 
iri7['N_Proyec_Estudios'] = np.where((iri7['Pendiente_Financiar'] >= 0)&(iri7['ESTADO SITUACIONAL']=='5. CON ESTUDIOS'), 1, 0) 
iri7['N_Proyec_Evaluacion'] = np.where((iri7['Pendiente_Financiar'] >= 0)&(iri7['ESTADO SITUACIONAL']=='4. EN EVALUACIÓN'), 1, 0) 
iri7['N_Proyec_Pendiente'] = np.where((iri7['Pendiente_Financiar'] >= 0)&(iri7['ESTADO SITUACIONAL']=='7. PENDIENTE'), 1, 0) 
iri7['N_Proyec_Asistencia_Tecnica'] = np.where((iri7['Pendiente_Financiar'] >= 0)&(iri7['ESTADO SITUACIONAL']=='6. ASISTENCIA TÉCNICA'), 1, 0) 
iri7['N_Proyec_Proceso_Contratación'] = np.where((iri7['Pendiente_Financiar'] >= 0)&(iri7['ESTADO SITUACIONAL']=='3. EN PROCESO DE CONTRATACIÓN'), 1, 0)

#Proyectos en Ejecución por Pliego
iri7['Nro_Proyectos_Pliego'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Pliego'].transform('sum'))
iri7['Nro_Proyectos_Ejecucion'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Ejecucion'].transform('sum'))
iri7['Nro_Proyec_Estudios'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Estudios'].transform('sum'))
iri7['Nro_Proyec_Evaluacion'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Evaluacion'].transform('sum'))
iri7['Nro_Proyec_Pendiente'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Pendiente'].transform('sum'))
iri7['Nro_Proyectos_Asistencia_Tecnica'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Asistencia_Tecnica'].transform('sum'))
iri7['Nro_Proyectos_Proceso_Contratacion'] = (iri7.groupby(['PLIEGO_SIAF_RES'])['N_Proyec_Proceso_Contratación'].transform('sum'))

#Proporción de Proyectos 
iri7['%Proyectos_Ejecucion'] = (iri7['Nro_Proyectos_Ejecucion'].fillna(0) / iri7['Nro_Proyectos_Pliego'].fillna(0))*100
iri7['%Proyec_Estudios'] = (iri7['Nro_Proyec_Estudios'].fillna(0) / iri7['Nro_Proyectos_Pliego'].fillna(0))*100
iri7['%Proyectos_Evaluacion'] = (iri7['Nro_Proyec_Evaluacion'].fillna(0) / iri7['Nro_Proyectos_Pliego'].fillna(0))*100
iri7['%Proyec_Pendiente'] = (iri7['Nro_Proyec_Pendiente'].fillna(0) / iri7['Nro_Proyectos_Pliego'].fillna(0))*100
iri7['%Proyectos_Asistencia_Tecnica'] = (iri7['Nro_Proyectos_Asistencia_Tecnica'].fillna(0) / iri7['Nro_Proyectos_Pliego'].fillna(0))*100
iri7['%Proyectos_Proceso_Contratacion'] = (iri7['Nro_Proyectos_Proceso_Contratacion'].fillna(0) / iri7['Nro_Proyectos_Pliego'].fillna(0))*100

#Avance por proyecto
iri7['PORC_AVANCE'] = iri7['DEV_ACUM_AL2023']/iri7['COSTO_ACTUAL_BCO']

#Avance por proyecto
iri7['Capacidad_Presupuestal'] = iri7['Pendiente_Financiar_Pliego'].fillna(0)/iri7['Prom_PIM_Pliego'].fillna(0)
iri7['Capacidad_Ejecucion'] = iri7['Pendiente_Financiar_Pliego'].fillna(0)/iri7['Prom_EJEC_Pliego'].fillna(0)

iri7.info()
iri7.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6476 entries, 0 to 6475
Data columns (total 74 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   CODIGO_UNICO                        6476 non-null   object 
 1   ENTIDAD_EJECUTORA                   1235 non-null   object 
 2   NIVEL_GOB_PCM                       1235 non-null   object 
 3   ESTADO SITUACIONAL                  6476 non-null   object 
 4   PLIEGO_SIAF_RES                     6474 non-null   object 
 5   NIVEL_GOB_VAL                       6476 non-null   object 
 6   VALIDACION_PLIEGO                   6476 non-null   object 
 7   SEC_EJEC                            6461 non-null   object 
 8   DEPARTAMENTO                        4955 non-null   object 
 9   PLIEGO_SIAF                         6461 non-null   object 
 10  ESTADO                              6476 non-null   object 
 11  SITUACION                           6476 no

,CODIGO_UNICO,ENTIDAD_EJECUTORA,NIVEL_GOB_PCM,ESTADO SITUACIONAL,PLIEGO_SIAF_RES,NIVEL_GOB_VAL,VALIDACION_PLIEGO,SEC_EJEC,DEPARTAMENTO,PLIEGO_SIAF,...,Nro_Proyectos_Proceso_Contratacion,%Proyectos_Ejecucion,%Proyec_Estudios,%Proyectos_Evaluacion,%Proyec_Pendiente,%Proyectos_Asistencia_Tecnica,%Proyectos_Proceso_Contratacion,PORC_AVANCE,Capacidad_Presupuestal,Capacidad_Ejecucion
0,2214514,MUNICIPALIDAD DISTRITAL DE HUARMACA,GOBIERNO LOCAL,2. EN EJECUCIÓN,200304. MD DE HUARMACA,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301551,20. PIURA,200304. MUNICIPALIDAD DISTRITAL DE HUARMACA,...,1.0,31.034483,65.517241,0.000000,0.0,0.0,3.448276,0.930747,0.171669,0.342671
1,2276731,MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS,GOBIERNO LOCAL,2. EN EJECUCIÓN,131101. MP DE GRAN CHIMU -CASCAS,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301205,13. LA LIBERTAD,131101. MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU...,...,2.0,27.272727,65.909091,2.272727,0.0,0.0,4.545455,0.019033,1.341038,3.296199
2,2280490,MUNICIPALIDAD DISTRITAL DE HUANCHACO,GOBIERNO LOCAL,3. EN PROCESO DE CONTRATACIÓN,130104. MD DE HUANCHACO,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301130,13. LA LIBERTAD,130104. MUNICIPALIDAD DISTRITAL DE HUANCHACO,...,1.0,28.571429,57.142857,0.000000,0.0,0.0,14.285714,0.902802,0.041946,0.153138
3,2286734,MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU - CASCAS,GOBIERNO LOCAL,2. EN EJECUCIÓN,131101. MP DE GRAN CHIMU -CASCAS,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,301205,13. LA LIBERTAD,131101. MUNICIPALIDAD PROVINCIAL DE GRAN CHIMU...,...,2.0,27.272727,65.909091,2.272727,0.0,0.0,4.545455,0.918567,1.341038,3.296199
4,2307424,MUNICIPALIDAD PROVINCIAL DE CUTERVO,GOBIERNO LOCAL,2. EN EJECUCIÓN,060601. MP DE CUTERVO,GOBIERNO LOCAL-1. GOBIERNO LOCAL-3. GOBIERNOS ...,SI,300605,06. CAJAMARCA,060601. MUNICIPALIDAD PROVINCIAL DE CUTERVO,...,0.0,11.111111,88.888889,0.000000,0.0,0.0,0.000000,0.848080,0.001676,0.002801


In [106]:
#%%
outputFile = os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto14', 'AVANCE_PIRCC_' +  fecha_corte + ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    iri7.to_excel(ew, sheet_name="DATA", index = False)   